In [1]:
!pip install opencv-python
!pip install opencv-python-headless

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
pip install -U scikit-image

Requirement already up-to-date: scikit-image in /usr/local/lib/python3.6/dist-packages (0.17.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import scipy

In [4]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras import layers, Sequential
from matplotlib import pyplot
from math import floor
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from numpy import expand_dims, array, exp, max
import json
import skimage.feature as feature
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
import sklearn
from sklearn.decomposition import PCA

from skimage.transform import resize

In [6]:
model1 = mobilenetv2()
model2 = Model(inputs=model1.inputs, outputs=model1.layers[1].output)
# model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
model = Sequential(layers=model2.layers)
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
# model = Model(inputs=model.inputs, outputs = model.layers[1].output)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 38, 32)        0         
Total params: 864
Trainable params: 864
Non-trainable params: 0
_________________________________________________________________


In [8]:
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
    self.numPoints = numPoints
    self.radius = radius

  def describe(self, image, eps = 1e-7):
    lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints+3), range=(0, self.numPoints + 2))

    # Normalize the histogram
    hist = hist.astype('float')
    hist /= (hist.sum() + eps)

    return hist, lbp

In [38]:
###########################THIS CODE CHECKS IMAGES FROM A FOLDER#################################

def sortTrainImages():
    # get the class labels from training datasets
    p = {}
    data_path = '../../StanfordDogs/Stanford_Dogs'
    img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255, rotation_range=20)
    #labels = img_gen.flow_from_directory(data_path + '/Images')
    #train_labels = labels.class_indices.keys()

    #for lbl in train_labels:
    resnet_path = data_path +'/Images/' + 'n02113978-Mexican_hairless'
    p_files = [f for f in listdir(resnet_path) if isfile(join(resnet_path, f))]
    pests_images = np.empty(len(p_files), dtype=object)
    print(p_files)
    for m in range(0, len(p_files)):
        
            #print(join(resnet_path, p_files[m]))
        imageName_x = p_files[m]
        p.update({'n02113978-Mexican_hairless' + "_" + str(m): imageName_x})
            ######if m < 25:
                #pests_images[m] = cv2.imread(join(resnet_path, p_files[m]))
                # get the name of the image
                ######imageName_x = p_files[m]
                #####p.update({lbl + "_" + str(m): imageName_x})

    return p


images_collection = sortTrainImages()

['n02113978_3391.jpg', 'n02113978_1535.jpg', 'n02113978_3504.jpg', 'n02113978_2276.jpg', 'n02113978_2727.jpg', 'n02113978_2979.jpg', 'n02113978_3349.jpg', 'n02113978_2141.jpg', 'n02113978_2677.jpg', 'n02113978_2446.jpg', 'n02113978_3823.jpg', 'n02113978_2546.jpg', 'n02113978_1999.jpg', 'n02113978_1854.jpg', 'n02113978_3197.jpg', 'n02113978_3714.jpg', 'n02113978_2261.jpg', 'n02113978_1030.jpg', 'n02113978_1595.jpg', 'n02113978_2167.jpg', 'n02113978_1238.jpg', 'n02113978_3056.jpg', 'n02113978_1391.jpg', 'n02113978_2128.jpg', 'n02113978_1957.jpg', 'n02113978_3375.jpg', 'n02113978_3734.jpg', 'n02113978_2425.jpg', 'n02113978_1954.jpg', 'n02113978_1823.jpg', 'n02113978_1620.jpg', 'n02113978_2804.jpg', 'n02113978_2143.jpg', 'n02113978_1867.jpg', 'n02113978_3220.jpg', 'n02113978_2709.jpg', 'n02113978_1970.jpg', 'n02113978_3134.jpg', 'n02113978_153.jpg', 'n02113978_3184.jpg', 'n02113978_605.jpg', 'n02113978_3655.jpg', 'n02113978_3474.jpg', 'n02113978_2306.jpg', 'n02113978_3249.jpg', 'n02113978_

In [39]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [40]:
#for testing scikit learn lbp implementation
# type of LBP
METHOD = 'uniform'  # at most two circular 0-1 or 1-0 transitions
radius = 1  # distance between central pixels and comparison pixels
n_points = 8 * radius  # define number of comparison pixels

#all features
all_features= []


In [ ]:
#second method
for o, j in images_collection.items():
    # create list for LBP representations
    lbp_imgs = []
    
    image_string = '../../StanfordDogs/Stanford_Dogs/Images/n02113978-Mexican_hairless/' + j
  
    img = imread(image_string)
    img = resize(img,(224,224,3))
        # get the path of image to extract the class
    img_class =j[0]
    img = img_to_array(img)    
    # expand the image dimensions
    img = expand_dims(img, axis=0)
        # scale the pixels
    img = preprocess_input(img)
        # get the features of the first layer
    feature_maps = model.predict(img)
   
    ix = 1
    square = 3
    features_prob_dist_array = []
    
    
    for _ in range(square):     
        
        each_main_feature = []
        
        for _ in range(square):
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            ##################FLATTEN THE FEATURE for dimensionality reduction##################################
            feature_items = array(feature_maps[0, :, :, ix - 1])
            #get the unsigned values of the feature
            features_x = feature_items.astype(np.uint8)
            lbp = local_binary_pattern(features_x, n_points, radius, METHOD)
            
            each_feature = []
            
            for h in lbp:
                #get the top 5 important features
                n_array = np.array(h)
                #each_sub_feature = np.prod(h, axis = 0, where = h > 0, keepdims = True)
                n_array= np.sort(n_array)
                each_soft = softMax(0.5,n_array[-8:])
                prod_sub_items = np.prod(each_soft)
                each_feature.append(prod_sub_items)
            numerator = each_feature
            denominator = np.trapz(numerator,axis=0)
            conf_list =(numerator/denominator)
            
            #final_array = softMax(0.5,features_prob_dist_array), the list is based on the softmax prob
            final_array_numerator=  (softMax(0.5,conf_list))
            final_array_denominator = np.trapz(final_array_numerator,axis=0)
            
            
            final_array_conf_list = final_array_numerator/final_array_denominator
            
            each_item_x = np.prod(final_array_conf_list)
            
            each_main_feature.append(each_item_x)
        
            #final_array_numerator=str(final_array_numerator)
            #first_chars = float(final_array_numerator[0:8])
            #print(first_chars)
            
            ix += 1
        #print(each_main_feature)   
        #get the conflation of all the features of the specific image
        
    #print(type(each_main_feature))
    
    new_rec = {j + "_/" +'n02113978-Mexican_hairless': each_main_feature}
    
    with open("../../files/mobilenetv2/lbp/stanford_target_domain_lbp.json", 'r+') as file:
            # First we load existing data into a dict.
            file_data = json.load(file)
            # Join new_data with file_data inside emp_details
            file_data["target_images"].append(new_rec)
            # Sets file's current position at offset.
            file.seek(0)
            # convert back to json.
            json.dump(file_data, file, indent=4)
    print("*")
       
    
       
        
    

*
*


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
